## 

# Crew AI - Book Search

## Import libraries

In [1]:
import os
from crewai import Crew, Task, Agent

# Warning control
import warnings
warnings.filterwarnings('ignore')

## Configuration

In [2]:
# ------ Ollama - Local---------#
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
embeddings_model = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS_MODEL"]=embeddings_model # documentar como definir variáveis de ambiente

# ------ OpenAI - API ---------#
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# ------ DEBUG ---------#
os.environ["DEBUG"]="False"
debug = os.getenv("DEBUG_MODE", "False")
if debug.lower() == "true":
    VERBOSE = 2
else:
    VERBOSE = False

### Crew - Busca no contexto

In [4]:
from langchain_openai import ChatOpenAI
from search.context import utils

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

from crewai_tools import tool

# Configurações locais
LOCAL_EMBEDDINGS_URL = "http://localhost:11434/api/embeddings"
EMBEDDINGS_MODEL = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS"] = LOCAL_EMBEDDINGS_URL
os.environ["LOCAL_EMBEDDINGS_MODEL"] = EMBEDDINGS_MODEL

# Configurações da API OpenAI
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Configurações de DEBUG
os.environ["DEBUG"] = "true"
DEBUG_MODE = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = 2 if DEBUG_MODE else False

# Constantes Globais
AVATARS = {"commentary": "./assets/e-book.png", "bible": "./assets/biblia.png",
           "revisor": "./assets/revisor.png"}
VECTOR_STORE = 'chroma'
PATH_BOOKS = f"./vector-store/{VECTOR_STORE}/books"
PATH_BIBLE = f"./vector-store/{VECTOR_STORE}/bible"
EMBEDDINGS_MODEL = os.getenv("LOCAL_EMBEDDINGS_MODEL")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
DEBUG = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = True #if DEBUG else False


# ------- TOOLS ------- #

@tool("Pesquisar em Livros")
def library_tool(text: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    db = utils.retriever_context(embeddings_model=EMBEDDINGS_MODEL, path_books=PATH_BOOKS, vector_store=VECTOR_STORE)
    documents = db.similarity_search(text)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source'].replace('./books/', '').replace('.pdf', '')
        page = row.metadata['page']
        result += f"**Trecho de {source} - página {page}:** \n{page_content}\n\n"
    return result

@tool("Pesquisa na Bíblia")
def bible_tool(text: str) -> str:
    """Retorna trechos bíblicos que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    db = utils.retriever_bible(embeddings_model=EMBEDDINGS_MODEL, path_books=PATH_BIBLE, vector_store=VECTOR_STORE)
    documents = db.similarity_search(text)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source'].replace('./bible/', '').replace('.pdf', '')
        page = row.metadata['page']
        result += f"**Trecho de {source} - página {page}:** \n{page_content}\n\n"
    return result


# ------- AGENTS ------- #

library_agent = Agent(
    role="Especialista em Pesquisa em Biblioteca",
    goal="Encontre os trechos de livros que falem explicitamente do tópico fornecido.",
    backstory=(
        "Você atualmente é encarregado de usar a library_tool "
        "para encontrar trechos que falem do tópico fornecido. "
        "Seu objetivo é garantir que os trechos que você fornece "
        "tratem diretamente do tópico e não uma relação distante. "
    ),
    tools=[library_tool],
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm
)

revisor_agent = Agent(
    role="Revisor de Texto",
    goal="Revisar e melhorar a qualidade dos trechos fornecidos pelos Especialistas.",
    backstory=(
        "Você atualmente é encarregado de revisar e melhorar a qualidade "
        "dos trechos fornecidos pelos Especialistas em Pesquisa da Biblioteca. "
        "Seu objetivo é garantir que os trechos fornecidos são relevantes, "
        "precisos e falem do tópico fornecido pelo cliente."
    ),
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm,
)

bible_agent = Agent(
    role="Especialista em Pesquisa Bíblica",
    goal="Encontre os trechos da Bíblia mais relevantes relacionados ao texto fornecido.",
    backstory=(
        "Você atualmente é encarregado de usar a bible_tool "
        "para encontrar trechos relevantes da Bíblia relacionados "
        "para um texto fornecido. "
        "Seu objetivo é garantir que os trechos que você fornece "
        "são altamente relevantes e úteis para análises posteriores. "
    ),
    tools=[bible_tool],
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm,
)

# ------- TASKS ------- #

library_search_task = Task(
    description=(
        "Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente "
        " do tópico: {text} "
    ),
    expected_output=(
        "Trechos de livros que falem explicitamente do tópico fornecido. "
        "Cada trecho deve ser claramente citado e incluída sua fonte, com o título do livro "
        "autor e número da página."
    ),
    agent=library_agent,
)

library_select_task = Task(
    description=(
        "Selecione os trechos que falam sobre o tópico: {text} "
        "Explique a relação ou indique caso não haja relação. Responda em português."
    ),
    expected_output=(
        "Tópico fornecido pelo cliente."
        "Trechos que falam do tópico fornecido pelo cliente, citando com sua fonte, incluindo o título do livro "
        "autor e número da página ou indicação caso não haja relação. "
        "Explicação onde o trecho fala diretamente sobre o tópico fornecido pelo cliente."
    ),
    agent=revisor_agent,
)

bible_search_task = Task(
    description=(
        "O cliente forneceu o seguinte texto para análise:\n"
        "{text}\n\n"
        "Sua tarefa é usar a bible_tool para encontrar os mais relevantes " 
        "trechos da Bíblia relacionados a este texto. "
    ),
    expected_output=(
        "Trechos da Bíblia diretamente relacionados ao texto fornecido. "
        "Cada trecho deve ser claramente citado com sua fonte, incluindo o título do livro "
        ", número da página e se possível capítulo e versículo. Os trechos devem ajudar "
        "a fornecer insights mais profundos ou contexto adicional ao texto fornecido."
    ),
    agent=bible_agent,
)


In [5]:
crew = Crew(
  agents=[library_agent, revisor_agent, bible_agent],
  tasks=[library_search_task, library_select_task, bible_search_task],
  verbose=2,
  memory=False,
)

In [6]:
input_text  = """O que falta aos pregadores hoje"""
input_text

'O que falta aos pregadores hoje'

In [7]:
inputs = {
    "text": input_text
}
result = crew.kickoff(inputs=inputs)

 [2024-06-25 23:48:55][DEBUG]: == Working Agent: Especialista em Pesquisa em Biblioteca
 [2024-06-25 23:48:55][INFO]: == Starting Task: Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente  do tópico: O que falta aos pregadores hoje 


> Entering new CrewAgentExecutor chain...
Vou começar pesquisando em livros o tópico fornecido para encontrar trechos que falem explicitamente sobre "O que falta aos pregadores hoje".

Action: Pesquisar em Livros
Action Input: {"text": "O que falta aos pregadores hoje"} 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era 

In [8]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_search_task))



**Tarefa:** 

Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente  do tópico: O que falta aos pregadores hoje  
 

**Saída:** 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Monk Kidd, "confiando que a migalha da graça cairá".102 Reconhecendo humildemente que são 
proletários impotentes para alcançarem o desejo  do coração sem ajuda di vina, eles são gratos 
pela menor migalha que cai da boca do pregador. 
Longas orações e palavras complicadas não convêm a maltrapilhos. Seu porta-voz é o 
cobrador de impostos no templo: "O Deus, sê propício a mim, pecador!" (Lc 18:13). O maltrapilho sabe que é o cobrad or de impostos e que recusar- se a admiti-lo faria dele um 
fariseu. Cobradores de impostos, zés-ninguém, vagabundos, mendigos de Jesus — os maltrapilhos sabem rir da sua vaidade de dese jarem ser percebidos e da vaidade de serem 
insignificantes.

**Trecho de Somos Todos Teólogos - RC Sproul - página 210:** 
Quando, porém, vier o Consolador, que eu vos enviarei da parte do Pai, o Espírito
da verdade, que dele procede, esse dará testemunho de mim; e vós também
testemunhareis, porque estais comigo desde o princípio.
Tenho-vos dito estas coisas para que não vos escandalizeis. Eles vos expulsarão
das sinagogas; mas vem a hora em que todo o que vos matar julgará com isso
tributar culto a Deus. Isto farão porque não conhecem o Pai, nem a mim. Ora, estas
coisas vos tenho dito para que, quando a hora chegar, vos recordeis de que eu vo-
las disse. Não vo-las disse desde o princípio, porque eu estava convosco (Jo 15.24-
16.4).
 
OUTRO CONSOLADOR

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 81:** 
vida e de amor que estão dentro de nós. E recai sobre nós o julgamento do Grande Senhor 
Executor de Mikado  sobre a garota que procrastinava perpe tuamente: "Jamais se sentirá falta 
dela; não, não: jamais se sentirá falta dela!" 
Cada um de nós paga um alto preço por nosso medo de cair de cara no chão. Ele assegura 
um estreitamento progressivo das nossas pe rsonalidades e impede a exploração e a 
experimentação. A medida que ficamos velhos seguimos fazendo apenas aquilo que fazemos 
bem. Não há crescimento em Jesus Cristo sem alguma dificuldade e embaraço. Se devemos nos 
manter crescendo, devemos permanecer correndo o risco de fracasso ao longo de toda a nossa


In [9]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_select_task))



**Tarefa:** 

Selecione os trechos que falam sobre o tópico: O que falta aos pregadores hoje Explique a relação ou indique caso não haja relação. Responda em português. 
 

**Saída:** 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
João era uma “Voz”. A maioria dos pregadores não passa de ecos, pois, se prestarmos bem atenção, saberemos dizer quais os livros que andaram lendo, e notaremos que citaram muito pouco do Livro. E hoje, só uma Voz, voz de um profeta enviado do céu para pregar aos pregadores, conseguiria despertar o coração dos homens. Só quem

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Longas orações e palavras complicadas não convêm a maltrapilhos. Seu porta-voz é o cobrador de impostos no templo: "O Deus, sê propício a mim, pecador!" (Lc 18:13). O maltrapilho sabe que é o cobrador de impostos e que recusar-se a admiti-lo faria dele um fariseu. Cobradores de impostos, zés-ninguém, vagabundos, mendigos de Jesus — os maltrapilhos sabem rir da sua vaidade de desejarem ser percebidos e da vaidade de serem insignificantes.

Explicação: Os trechos selecionados falam sobre o tópico "O que falta aos pregadores hoje" ao abordarem a questão da autenticidade e profundidade da mensagem pregada. O primeiro trecho destaca a necessidade de pregadores serem mais do que simples "ecos" de outros pregadores, apontando para a importância de uma voz autêntica e profunda para despertar os corações dos ouvintes. Já o segundo trecho ressalta a humildade e simplicidade necessárias na pregação, destacando a importância de reconhecer a própria condição de pecador e a necessidade da graça divina. Ambos os trechos abordam aspectos essenciais que muitas vezes faltam nos pregadores contemporâneos.


In [10]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(bible_search_task))



**Tarefa:** 

O cliente forneceu o seguinte texto para análise:
O que falta aos pregadores hoje

Sua tarefa é usar a bible_tool para encontrar os mais relevantes trechos da Bíblia relacionados a este texto.  
 

**Saída:** 

Jeremias 23:1-2 - "Ai dos pastores que destroem e dispersam as ovelhas do meu pasto! diz o Senhor. Portanto, assim diz o Senhor Deus de Israel, contra os pastores que apascentam o meu povo: Vós dispersastes as minhas ovelhas, e as afugentastes, e não as visitastes; eis que visitarei sobre vós a maldade das vossas ações, diz o Senhor."


In [11]:
Markdown(result)

Jeremias 23:1-2 - "Ai dos pastores que destroem e dispersam as ovelhas do meu pasto! diz o Senhor. Portanto, assim diz o Senhor Deus de Israel, contra os pastores que apascentam o meu povo: Vós dispersastes as minhas ovelhas, e as afugentastes, e não as visitastes; eis que visitarei sobre vós a maldade das vossas ações, diz o Senhor."

In [12]:
print(crew.usage_metrics)

{'total_tokens': 6800, 'prompt_tokens': 5034, 'completion_tokens': 1766, 'successful_requests': 5}


In [13]:
#todo: criar um script python para rodar e retornar tudo isso depois desenvolver o contexto e relações bíblicas contexto histórico etc